<a href="https://colab.research.google.com/github/nalpata/proyecto_aplicado_preservantes/blob/main/Proyecto_1_Hito_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Baseline - Proyecto Aplicado: Preservantes

En este notebook construimos el baseline de un sistema RAG usando un conjunto de PDFs
sobre preservantes. Incluye:

1. Carga e ingesta de PDFs
2. Preprocesamiento básico y chunking
3. Generación de embeddings
4. Creación de un vector store
5. Retriever (similarity search)
6. Benchmark (Precision@k sobre un set de preguntas)


In [2]:
## Instalación de librerías (celda de código)
!pip install -q langchain langchain-community langchain-text-splitters \
               chromadb sentence-transformers pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.1 MB/s eta 0:0

In [3]:
##Importaciones y configuración básica
import os
from pathlib import Path

# LangChain imports
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Para evaluación básica
from typing import List, Dict
import numpy as np

# Para ver resultados
from pprint import pprint


In [4]:
# RESETEAR TODO PARA PARTIR LIMPIO EN COLAB

import os, shutil

# 1) Ir a /content
%cd /content

# 2) Borrar cualquier clone previo duplicado
if os.path.exists("proyecto_aplicado_preservantes"):
    shutil.rmtree("proyecto_aplicado_preservantes")
    print("🗑️ Carpeta borrada: proyecto_aplicado_preservantes")

# 3) Clonar de nuevo desde tu GitHub
!git clone https://github.com/nalpata/proyecto_aplicado_preservantes.git

# 4) Entrar a la carpeta correcta
%cd proyecto_aplicado_preservantes

print("\n🎉 Listo. Ahora estamos en el repo correcto sin duplicados.")
!ls


/content
Cloning into 'proyecto_aplicado_preservantes'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 121 (delta 39), reused 77 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 19.34 MiB | 37.08 MiB/s, done.
Resolving deltas: 100% (39/39), done.
/content/proyecto_aplicado_preservantes

🎉 Listo. Ahora estamos en el repo correcto sin duplicados.
 02_chunking.ipynb	    MEJORAS_CHUNKING.md    RESUMEN_PROBLEMAS.md
 CHECKLIST_INSTALACION.md   notebooks		   run_pipeline.py
 data			   'Pauta proyecto.pdf'    SOLUCION_INSTALACION.md
 DIAGRAMAS_HITO_1.md	    PROBLEMAS_COMUNES.md   src
 examples.py		    QUICK_START.md	   streamlit_app.py
 FIX_CHROMADB.md	    README.md		   test_improvements.py
 install_dependencies.sh    requirements.txt	   TESTING_LOCAL.md
 install_fix.sh		    RESUMEN_FINAL.md	   test_single_pdf.py
 INSTRUCCIONES_PRUEBA.md    RESUMEN_HITO_1.md


In [5]:
# Ruta base del proyecto en Colab
BASE_PATH = Path("/content/proyecto_aplicado_preservantes")

DATA_PDF_DIR = BASE_PATH / "data" / "pdfs"          # aquí PDFs de preservantes
CHROMA_DIR   = BASE_PATH / "chroma_preservantes"   # carpeta donde se guardará el vector store

BASE_PATH.mkdir(parents=True, exist_ok=True)
CHROMA_DIR.mkdir(parents=True, exist_ok=True)

print("Base path:", BASE_PATH)
print("PDF dir:", DATA_PDF_DIR)
print("Chroma dir:", CHROMA_DIR)


Base path: /content/proyecto_aplicado_preservantes
PDF dir: /content/proyecto_aplicado_preservantes/data/pdfs
Chroma dir: /content/proyecto_aplicado_preservantes/chroma_preservantes


In [6]:
##Carga de documentos (ingesta de PDFs)
def load_pdfs(pdf_dir: Path):
    """
    Carga todos los PDFs de una carpeta usando LangChain.
    Devuelve una lista de Documents.
    """
    loader = DirectoryLoader(
        str(pdf_dir),
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        show_progress=True
    )
    docs = loader.load()
    return docs

raw_docs = load_pdfs(DATA_PDF_DIR)
len(raw_docs), raw_docs[0]


100%|██████████| 19/19 [00:25<00:00,  1.33s/it]


(475,
 Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'Elsevier', 'creationdate': '2020-01-24T15:54:21+05:30', 'subject': 'Introduction to the Chemistry of Food 251-311. 10.1016/B978-0-12-809434-1.00007-4', 'elsevierwebpdfspecifications': '6.5', 'robots': 'noindex', 'moddate': '2020-01-28T18:17:41+05:30', 'doi': '10.1016/B978-0-12-809434-1.00007-4', 'title': 'Chapter Seven - Food additives', 'source': '/content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf', 'total_pages': 61, 'page': 0, 'page_label': '251'}, page_content='CHAPTER SEVEN\nFood additives\nLearning objectives\nThis chapter will help you to:\n· Describe FDAs role in regulation of food additives\n· Deﬁne the terms natural and synthetic additives\n· Describe uses for food additives\n· Deﬁne the term hydrocolloid and describe uses in creating novel foods\n· Explain the difference between toxins and toxicants\nIntroduction\nFood additives are employed for a variety of purposes. The

In [7]:
##Preprocesamiento
def clean_metadata(docs):
    """
    Normaliza  los metadatos: agrega un campo 'source'
    y mantiene solo lo relevante.
    """
    cleaned = []
    for d in docs:
        meta = d.metadata or {}
        source = meta.get("source", "")
        # Nos quedamos con un metadata simple
        new_meta = {
            "source": source,
            "page": meta.get("page", None)
        }
        d.metadata = new_meta
        cleaned.append(d)
    return cleaned

docs = clean_metadata(raw_docs)
len(docs), docs[0].metadata


(475,
 {'source': '/content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf',
  'page': 0})

**Chunking baseline**

Usamos RecursiveCharacterTextSplitter con:
- chunk_size = 800
- chunk_overlap = 200



In [9]:
##Chunking
##Usamos RecursiveCharacterTextSplitter como baseline.

CHUNK_SIZE = 800      # puedes jugar con este valor
CHUNK_OVERLAP = 200   # superposición para mantener contexto

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

chunks = text_splitter.split_documents(docs)
len(chunks), chunks[0]


(2790,
 Document(metadata={'source': '/content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf', 'page': 0}, page_content='CHAPTER SEVEN\nFood additives\nLearning objectives\nThis chapter will help you to:\n· Describe FDAs role in regulation of food additives\n· Deﬁne the terms natural and synthetic additives\n· Describe uses for food additives\n· Deﬁne the term hydrocolloid and describe uses in creating novel foods\n· Explain the difference between toxins and toxicants\nIntroduction\nFood additives are employed for a variety of purposes. They are used\nto improve nutrition, enhance safety, add color andﬂavor, modify texture,\nand preserve food. Some food additives contribute more than one function.\nAcids, for example, give food a tangyﬂavor, but also inhibit the growth of\nmicroorganisms that cause spoilage. Nitrite also has multiple functions. It is'))

In [39]:
##cuántos PDFs y de qué archivo vienen los chunks
from collections import Counter

print("N° de documentos originales:", len(raw_docs))
print("Fuentes (PDFs) originales:")
for src in sorted({d.metadata.get("source") for d in raw_docs}):
    print(" -", src)

print("\nN° de chunks:", len(chunks))
print("N° de chunks por PDF:")
conteo = Counter(d.metadata.get("source") for d in chunks)
for src, c in conteo.items():
    print(f"{src}: {c}")


N° de documentos originales: 475
Fuentes (PDFs) originales:
 - /content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2212429225020413-main.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/1-s2.0-S2405844023042287-main.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/AntimicrobialActivityofSpiceextracts.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Effects of Acidification and Preservatives on Microbial Growth Puree.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/FTB-61-212.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/IJFS2018-8410747.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Molecules to food, preservatives.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Preservantes.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf
 - /content/proyecto_aplicado_preservantes/data/pdfs/The Scientific World Journal - 2022 - Tes

Usamos chunking gerarquico porque los chunks no estan balanceados y se genera un pdf dominante

In [40]:
## Uso chunking gerarquico el chunking simple siempre retorna un solo pdf
from langchain_text_splitters import RecursiveCharacterTextSplitter
from uuid import uuid4

# Splitter de nivel alto (bloques grandes)
high_level_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
)

# Splitter de nivel bajo (para el vector store)
low_level_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=150,
    length_function=len,
)

def hierarchical_chunk(docs):
    """
    1. Divide en bloques grandes (nivel 1)
    2. Cada bloque grande se subdivide en chunks pequeños (nivel 2)
    3. Añade metadatos de jerarquía (parent_id, level1_index)
    """
    level1_docs = high_level_splitter.split_documents(docs)

    final_chunks = []
    for idx, d in enumerate(level1_docs):
        parent_id = str(uuid4())  # id único del bloque grande

        # subdividir este bloque
        sub_docs = low_level_splitter.split_documents([d])

        for s in sub_docs:
            meta = dict(s.metadata)
            meta["parent_id"] = parent_id
            meta["level1_index"] = idx
            s.metadata = meta
            final_chunks.append(s)

    return final_chunks

hier_chunks = hierarchical_chunk(docs)
len(hier_chunks), hier_chunks[0].metadata


(3519,
 {'source': '/content/proyecto_aplicado_preservantes/data/pdfs/Food aditives1.pdf',
  'page': 0,
  'parent_id': '0642451e-ef62-4822-b12b-cbfad146373c',
  'level1_index': 0})

In [41]:
from pathlib import Path
from langchain_community.vectorstores import Chroma

BASE_PATH = Path("/content/proyecto_aplicado_preservantes")
CHROMA_HIER_DIR = BASE_PATH / "chroma_preservantes_hier"

import shutil
shutil.rmtree(CHROMA_HIER_DIR, ignore_errors=True)
CHROMA_HIER_DIR.mkdir(parents=True, exist_ok=True)

vector_store_hier = Chroma.from_documents(
    documents=hier_chunks,
    embedding=embeddings,                  # el mismo modelo multilingüe que ya estás usando
    persist_directory=str(CHROMA_HIER_DIR)
)

vector_store_hier.persist()
print("Vector store HIER creado con", vector_store_hier._collection.count(), "documentos")


Vector store HIER creado con 3519 documentos


In [42]:
retriever_hier = vector_store_hier.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 20}
)


In [43]:
def inspeccionar_query_con(retriever, query: str, k: int = 5):
    docs = retriever.invoke(query)[:k]
    print("Query:", query, "\n")
    for i, d in enumerate(docs, 1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"),
              "| level1_index:", d.metadata.get("level1_index"))
        print(d.page_content[:500], "...\n")

inspeccionar_query_con(retriever_hier, "¿Qué se entiende por vida útil de un alimento?")


Query: ¿Qué se entiende por vida útil de un alimento? 

--- Documento 1 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 4 | level1_index: 521
constante que me ha permitido ser una persona de bien, gracias por ayudarme y  
brindarme los recursos necesarios para cumplir mis objetivos como persona y  
estudiante. 
A  mis  abuelitos  quienes  estuvieron  junto  a  mí  incondicionalmente  en  los 
momentos más difíciles de mi vida al igual que toda mi familia.
A Maribel Cerón con quien compartí mi vida universitaria, y representó gran  
esfuerzo y tesón en momentos de decline y cansancio, por estar  siempre en las  
buenas y en las malas,  ...

--- Documento 2 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 46 | level1_index: 575
aditivos,  los  cuales  según  la  Academia  Nacional  de  Ciencias  son  aquellos 
compuestos  que  son  incorporados  a  los  alimentos  de  manera  directa  o 
indirect

**Modelo de embeddings**

In [44]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME
)


**Vector Store**

In [45]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=str(CHROMA_DIR)
)

vector_store.persist()
print("Vector store creado con", vector_store._collection.count(), "documentos")


Vector store creado con 5580 documentos


**Retriever simple**

In [15]:
# Retriever simple: similarity search con k documentos
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

query_ejemplo = "¿Qué es un preservante y qué función cumple en alimentos?"
resultados = retriever.invoke(query_ejemplo)

len(resultados), resultados[0]

(5,
 Document(metadata={'source': '/content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf', 'page': 16}, page_content='una serie de nuevos conocimientos sin los cuales no sería posible afrontarla.  \nSin embargo, no deja de ser un campo muy nuevo en lo que respecta a la  \ninvestigación. \nLas  industrias  de  alimentos  son  las  principales  interesadas  en  controlar  la \npoblación  microbiana  que  puedan  alterar  o  dañar  las  características  del \nalimento, de manera que sustancias como el propionato de calcio y sorbato de  \npotasio son empleadas como conservantes. Sin embargo el empleo de este  \ntipo de sustancias debe depender de las características del alimento, y de los  \nefectos que pueda ejercer el preservante en el producto, como olor, sabor y  \ntextura, además de efectos secundarios en los consumidores. De modo que la'))

In [16]:
for i, d in enumerate(resultados, 1):
    print(f"\n### Documento {i} ###")
    print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"))
    print(d.page_content[:300], "...")



### Documento 1 ###
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 16
una serie de nuevos conocimientos sin los cuales no sería posible afrontarla.  
Sin embargo, no deja de ser un campo muy nuevo en lo que respecta a la  
investigación. 
Las  industrias  de  alimentos  son  las  principales  interesadas  en  controlar  la 
población  microbiana  que  puedan  alterar  ...

### Documento 2 ###
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 21
realidad  es  única  y  fragmentable  en  partes  que  se  pueden  manipular 
independientemente, y la relación sujeto-objeto es independiente. Para este  
enfoque  la  realidad  es  algo  exterior,  ajeno,  objetivo  y  puede  y  debe  ser 
estudiada y por tanto conocida.
2.2.1  Mercado de producto ...

### Documento 3 ###
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 21
proyecto: Aplicación de una formulación inade

**Naive RAG**

In [19]:
##Baseline: solo mostrar textos recuperados
def show_retrieval(query: str, k: int = 5, retriever=retriever):
    # Con LangChain nuevo el retriever se invoca así:
    docs = retriever.invoke(query)   # <-- aquí está el cambio
    docs = docs[:k]

    print(f"Query: {query}\n")
    for i, d in enumerate(docs, start=1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "Page:", d.metadata.get("page"))
        print(d.page_content[:500], "...")
        print()

# Prueba
show_retrieval("Tipos de preservantes utilizados en bebidas")


Query: Tipos de preservantes utilizados en bebidas

--- Documento 1 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf Page: 16
una serie de nuevos conocimientos sin los cuales no sería posible afrontarla.  
Sin embargo, no deja de ser un campo muy nuevo en lo que respecta a la  
investigación. 
Las  industrias  de  alimentos  son  las  principales  interesadas  en  controlar  la 
población  microbiana  que  puedan  alterar  o  dañar  las  características  del 
alimento, de manera que sustancias como el propionato de calcio y sorbato de  
potasio son empleadas como conservantes. Sin embargo el empleo de este  
tipo de su ...

--- Documento 2 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf Page: 57
Descriptivo: Su finalidad es explicar el comportamiento de  una variable en  
función de otras; aquí se plantea una relación de causa y tiene que cumplir  
otros  criterios  de  causalidad  (Bradfor  Hill);  requiere  

**Integramos un LLM para responder**

In [23]:
!pip install -q langchain-openai langchain-community openai tiktoken


In [25]:
!pip install -q langchain langchain-openai langchain-community langchain-text-splitters
!pip install -q langchain-core
!pip install -q langchain-experimental
!pip install -q langchainhub
!pip install -q lc-retrieval


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.19.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.19.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement lc-retrieval (from versions: none)
ERROR: No matching distribution found for lc-retrieval


In [28]:
import os
import getpass

from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough


In [29]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingresa tu OPENAI_API_KEY: ")


Ingresa tu OPENAI_API_KEY: ··········


In [30]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)


In [32]:
def ask_rag(query: str, k: int = 5):
    # 1. Recuperar documentos relevantes
    docs = retriever.invoke(query)
    docs = docs[:k]

    # 2. Construir el contexto a partir de los chunks
    context = "\n\n---\n\n".join(d.page_content for d in docs)

    # 3. Armar el prompt para el LLM
    prompt = f"""
Eres un asistente experto en preservantes de alimentos.
Responde usando EXCLUSIVAMENTE la información del contexto.

Contexto:
{context}

Pregunta: {query}

Respuesta en español, clara y concisa:
"""

    # 4. Llamar al modelo
    response = llm.invoke(prompt)

    # 5. Mostrar resultado y fuentes
    print("Pregunta:", query)
    print("\n📌 Respuesta:\n")
    print(response.content)

    print("\n📎 Fuentes:")
    for d in docs:
        print("-", d.metadata.get("source"), "page", d.metadata.get("page"))


In [33]:
ask_rag("¿Qué tipos de preservantes se usan en bebidas?")


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

**Benchmark para el baseline (Precision@k)**

In [46]:
# Dataset de evaluación manual
eval_queries = [
    {
        "query": "¿Qué es un preservante antimicrobiano?",
        "relevant_keywords": [
            # español
            "preservante antimicrobiano",
            "conservante antimicrobiano",
            "inhibir el crecimiento de microorganismos",
            "inhibición de microorganismos",
            # inglés
            "antimicrobial preservative",
            "antimicrobial agent",
            "inhibit growth of microorganisms",
            "inhibits microbial growth"
        ]
    },
    {
        "query": "¿Cuáles son los factores que afectan la efectividad de los preservantes?",
        "relevant_keywords": [
            # español
            "factores que afectan la efectividad",
            "eficacia de los preservantes",
            "pH",
            "actividad de agua",
            "aw",
            "temperatura",
            "composición del alimento",
            "concentración del preservante",
            "envase",
            # inglés
            "factors affecting effectiveness",
            "factors influencing efficacy",
            "water activity",
            "food composition",
            "preservative concentration",
            "storage temperature",
            "packaging"
        ]
    },
    {
        "query": "¿Qué se entiende por vida útil de un alimento?",
        "relevant_keywords": [
            # español
            "vida útil",
            "vida-útil",
            "tiempo de conservación",
            "periodo de conservación",
            "duración de la vida útil",
            "tiempo durante el cual el alimento",
            # inglés
            "shelf life",
            "storage life",
            "time during which the food",
            "period during which the food",
            "expiry date",
            "expiration date"
        ]
    },
]

In [47]:
import numpy as np

def precision_at_k(
    retriever,
    query: str,
    relevant_keywords,
    k: int = 5
) -> float:
    """
    Calcula Precision@k para una query:
    - Usa retriever.invoke(query) (versión nueva de LangChain)
    - Considera acierto si el chunk contiene alguna palabra clave
    """

    docs = retriever.invoke(query)
    docs = docs[:k]

    hits = 0
    for d in docs:
        text = d.page_content.lower()
        if any(kw.lower() in text for kw in relevant_keywords):
            hits += 1

    return hits / k if k > 0 else 0.0


def evaluate_retriever(eval_queries, k: int = 5):
    """
    Recorre una lista de queries de evaluación y
    muestra Precision@k por pregunta y el promedio.
    """
    scores = []

    for item in eval_queries:
        p_at_k = precision_at_k(
            retriever=retriever,
            query=item["query"],
            relevant_keywords=item["relevant_keywords"],
            k=k
        )
        scores.append(p_at_k)

        print(f"Query: {item['query']}")
        print(f"Precision@{k}: {p_at_k:.2f}")
        print()

    print(f"Precision@{k} promedio: {np.mean(scores):.2f}")
    return scores


# Ejecutar evaluación
scores = evaluate_retriever(eval_queries, k=5)


Query: ¿Qué es un preservante antimicrobiano?
Precision@5: 0.00

Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes?
Precision@5: 0.00

Query: ¿Qué se entiende por vida útil de un alimento?
Precision@5: 0.00

Precision@5 promedio: 0.00


In [48]:
def inspeccionar_query(query: str, k: int = 5):
    docs = retriever.invoke(query)[:k]
    print("Query:", query, "\n")
    for i, d in enumerate(docs, 1):
        print(f"--- Documento {i} ---")
        print("Source:", d.metadata.get("source"), "| Page:", d.metadata.get("page"))
        print(d.page_content[:600], "...\n")

inspeccionar_query("¿Cuáles son los factores que afectan la efectividad de los preservantes?")


Query: ¿Cuáles son los factores que afectan la efectividad de los preservantes? 

--- Documento 1 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 16
una serie de nuevos conocimientos sin los cuales no sería posible afrontarla.  
Sin embargo, no deja de ser un campo muy nuevo en lo que respecta a la  
investigación. 
Las  industrias  de  alimentos  son  las  principales  interesadas  en  controlar  la 
población  microbiana  que  puedan  alterar  o  dañar  las  características  del 
alimento, de manera que sustancias como el propionato de calcio y sorbato de  
potasio son empleadas como conservantes. Sin embargo el empleo de este  
tipo de sustancias debe depender de las características del alimento, y de los  
efectos que pueda ejercer el  ...

--- Documento 2 ---
Source: /content/proyecto_aplicado_preservantes/data/pdfs/Prop Ca y s. potasio.pdf | Page: 16
una serie de nuevos conocimientos sin los cuales no sería posible afrontarla.  
Sin 